In [6]:
def create_vaild_item_store_combinations(_df):
    df = _df.copy()
    df['log1p'] = np.log(df['units'] + 1)
    
    g = df.groupby(["store_nbr", "item_nbr"])['log1p'].mean()
    g = g[g > 0.0]
    
    store_nbrs = g.index.get_level_values(0)
    item_nbrs = g.index.get_level_values(1)
    
    store_item_nbrs = sorted(zip(store_nbrs, item_nbrs), key = lambda t: t[1] * 10000 + t[0] )
    return store_item_nbrs
df_train = pd.read_csv("../asset/train.csv")
tuple_list = create_vaild_item_store_combinations(df_train)

In [32]:
df_train = pd.read_csv("../asset/train.csv")
df = df_train.copy()
df['log1p'] = np.log(df['units'] + 1)

# g = df.groupby(["store_nbr", "item_nbr"])['log1p'].mean()
# g = g[g > 0.0]

# store_nbrs = g.index.get_level_values(0)
# item_nbrs = g.index.get_level_values(1)

# store_item_nbrs = sorted(zip(store_nbrs, item_nbrs), key = lambda t: t[1] * 10000 + t[0] )

In [34]:
g = df.groupby(["store_nbr", "item_nbr"])['log1p'].mean()

In [37]:
g = g[g > 0.0]

In [33]:
df.tail()

,date,store_nbr,item_nbr,units,log1p
4617595,2014-10-31,45,107,0,0.0
4617596,2014-10-31,45,108,0,0.0
4617597,2014-10-31,45,109,0,0.0
4617598,2014-10-31,45,110,0,0.0
4617599,2014-10-31,45,111,0,0.0


# 0번 컬럼은 스토어 넘버, 1번은 아이템 넘버(매출이 있는 것만 들어감)

In [11]:
store_item_nbrs = pd.DataFrame(tuple_list, columns=["store_nbr", "item_nbr"])

In [29]:
store_item_nbrs[store_item_nbrs["store_nbr"] == 45]

,store_nbr,item_nbr
5,45,4
43,45,9
68,45,22
74,45,26
82,45,34
131,45,50


In [13]:
import pickle
import os.path

def get_holidays(fpath):
    # holidays are from http://www.timeanddate.com/holidays/us/ , holidays and some observances
    
    f = open(fpath)
    lines = f.readlines()
    lines = [line.split(" ")[:3] for line in lines]
    lines = ["{} {} {}".format(line[0], line[1], line[2]) for line in lines]
    lines = pd.to_datetime(lines)
    return pd.DataFrame({"date2":lines})

def get_holiday_names(fpath):
    # holiday_names are holidays + around Black Fridays
    
    f = open(fpath)
    lines = f.readlines()
    lines = [line.strip().split(" ")[:4] for line in lines]
    lines_dt = ["{} {} {}".format(line[0], line[1], line[2]) for line in lines]
    lines_dt = pd.to_datetime(lines_dt)
    lines_hol = [line[3] for line in lines]
    return pd.DataFrame({"date2":lines_dt, "holiday_name":lines_hol})

def to_float(series, replace_value_for_M, replace_value_for_T):
    series = series.map(lambda s : s.strip())
    series[series == 'M'] = replace_value_for_M
    series[series == 'T'] = replace_value_for_T
    return series.astype(float)

def preprocess(_df, is_train):
    
    df = _df.copy()

    # log1p
    if is_train: 
        df['log1p'] = np.log(df['units'] + 1)

    # date
    df['date2'] = pd.to_datetime(df['date'])

    # weather features
    wtr['date2'] = pd.to_datetime(wtr.date)
    wtr["preciptotal2"] = to_float(wtr["preciptotal"], 0.00, 0.005)
    wtr["preciptotal_flag"] = np.where(wtr["preciptotal2"] > 0.2, 1.0, 0.0)

    wtr["depart2"] = to_float(wtr.depart, np.nan, 0.00)
    wtr["depart_flag"] = 0.0
    wtr["depart_flag"] = np.where(wtr["depart2"] < -8.0, -1, wtr["depart_flag"])
    wtr["depart_flag"] = np.where(wtr["depart2"] > 8.0 ,  1, wtr["depart_flag"])
    df = pd.merge(df, key, on='store_nbr')
    df = pd.merge(df, wtr[["date2", "station_nbr", "preciptotal_flag", "depart_flag"]], 
                      on=["date2", "station_nbr"])
    
    # weekday
    df['weekday'] = df.date2.dt.weekday
    df['is_weekend'] = df.date2.dt.weekday.isin([5,6])
    df['is_holiday'] = df.date2.isin(holidays.date2)
    df['is_holiday_weekday'] = df.is_holiday & (df.is_weekend == False)
    df['is_holiday_weekend'] = df.is_holiday &  df.is_weekend

    # bool to int (maybe no meaning)
    df.is_weekend = np.where(df.is_weekend, 1, 0)
    df.is_holiday = np.where(df.is_holiday, 1, 0)
    df.is_holiday_weekday = np.where(df.is_holiday_weekday, 1, 0)
    df.is_holiday_weekend = np.where(df.is_holiday_weekend, 1, 0)
    
    # day, month, year
    df['day'] = df.date2.dt.day
    df['month'] = df.date2.dt.month
    df['year'] = df.date2.dt.year
    
    # around BlackFriday
    df = pd.merge(df, holiday_names, on='date2', how = 'left')
    df.loc[df.holiday_name.isnull(), "holiday_name"] = ""

    around_BlackFriday = ["BlackFridayM3", "BlackFridayM2", "ThanksgivingDay", "BlackFriday",
                          "BlackFriday1", "BlackFriday2", "BlackFriday3"]
    df["around_BlackFriday"] = np.where(df.holiday_name.isin(around_BlackFriday), 
                                        df.holiday_name, "Else")

    return df

# read dataframes
key = pd.read_csv("../asset/key.csv")
wtr = pd.read_csv("../asset/weather.csv")
holidays = get_holidays("holidays.txt")
holiday_names = get_holiday_names("holiday_names.txt")

valid_store_items = store_item_nbrs

# preprocess 
df_train = pd.read_csv("../asset/train.csv")
mask_train = [(sno_ino in valid_store_items) for sno_ino in zip(df_train['store_nbr'], df_train['item_nbr']) ]
print(mask_train)
df_train = df_train[mask_train].copy()
preprocess(train_preprocess, True).to_pickle('model/train2.pkl')

df_test =  pd.read_csv("../asset/test.csv")
mask_test = [(sno_ino in valid_store_items) for sno_ino in zip(df_test['store_nbr'], df_test['item_nbr']) ]
df_test =  df_test[mask_test].copy()
preprocess(df_test,  False).to_pickle('model/test2.pkl')

In [40]:
li = [sno_ino for sno_ino in zip(df_train['store_nbr'], df_train['item_nbr'])]

In [62]:
li_ = [sno_ino in tuple_list for sno_ino in li]

In [63]:
li_[:5]

[False, False, False, False, False]

In [44]:
len(df_train)

236038

In [52]:
df_train.to_pickle('excluded_zero_units.pkl')

In [49]:
from datetime import datetime

In [50]:
def saveDataFrameToCsv(df, fileName, idx = False):
    """
        넘겨준 df를 filename + 년월일시간분 의 format으로 이루어진 이름의 파일로 생성해준다.
        index를 True로 넘겨주면 저장할 때 아규먼트로 index=True를 넣어주게 된다.
    """
    fileName += "_" + datetime.now().strftime("%Y%m%d%H%M") + ".csv"
    return df.to_csv(fileName, index = idx)

In [51]:
saveDataFrameToCsv(df_train, "excluded_zero_units")

In [53]:
train_preprocess = pickle.load(open("excluded_zero_units.pkl", "rb"))

In [55]:
train_preprocess.tail()

,date,store_nbr,item_nbr,units
4617497,2014-10-31,45,9,10
4617510,2014-10-31,45,22,0
4617514,2014-10-31,45,26,1
4617522,2014-10-31,45,34,0
4617538,2014-10-31,45,50,0


In [56]:
preprocess(train_preprocess, True).to_pickle('model/train2.pkl')

In [59]:
train_data = pickle.load(open("model/train2.pkl", "rb"))

In [60]:
train_data.tail()

,date,store_nbr,item_nbr,units,log1p,date2,station_nbr,preciptotal_flag,depart_flag,weekday,is_weekend,is_holiday,is_holiday_weekday,is_holiday_weekend,day,month,year,holiday_name,around_BlackFriday
236033,2014-10-31,35,50,0,0.000000,2014-10-31,5,0.0,0.0,4,0,1,1,0,31,10,2014,Halloween,Else
236034,2014-10-31,35,63,0,0.000000,2014-10-31,5,0.0,0.0,4,0,1,1,0,31,10,2014,Halloween,Else
236035,2014-10-31,35,66,10,2.397895,2014-10-31,5,0.0,0.0,4,0,1,1,0,31,10,2014,Halloween,Else
236036,2014-10-31,35,93,0,0.000000,2014-10-31,5,0.0,0.0,4,0,1,1,0,31,10,2014,Halloween,Else
236037,2014-10-31,35,105,0,0.000000,2014-10-31,5,0.0,0.0,4,0,1,1,0,31,10,2014,Halloween,Else


In [61]:
train_data.to_pickle('preprocess_date.pkl')

In [ ]:
train_13 = pd.concat([train, station_13] , axis = 1 , join = 'inner' )